# Welcome to the Petabytes-3 WWT vizualization tutorial!

Here are some helpful links:

- [The tutorial landing page](https://wwt-forum.org/t/researcher-workshop-wwt-petabytes-to-science-3-boston-usa-2019-nov-8/86/2)
- [The pywwt documentation](https://pywwt.readthedocs.io/)

# Activity 0: Starting up pywwt

The first thing to do is to open up a pywwt window in this notebook session. As is generally the case, we have to start with some Python imports:

In [ ]:
from pywwt.jupyter import WWTJupyterWidget

We'll also set up a utility function:

In [ ]:
def datapath(*args):
    from os.path import join
    return join('..', 'data', *args)

Next, create a widget and display it inline. (That's why the final line is a bare `wwt`.)

In [ ]:
wwt = WWTJupyterWidget()
wwt

*It might look like you got a black screen but it's probably OK.* Click your mouse in the widget view and drag to either side. You should see the Milky Way scroll into view! Check with a neighbor if you're concerned that things are not working.

Next, we will move the WWT view to a new window pane, which makes it much easier to interact with the visualization while writing code:

1. Move your mouse *to the left side of the WWT widget area* and right-click. You should get a tall pop-up menu.
2. Choose "Create New View For Output". A new WWT window should open up.
3. Still in this notebook pane, left-click on the blue bar next to the WWT widget area. It should hide.
4. If you like, click-and-drag the notebook tab that says "Output View" to move it to a different area of the screen. I like it at the top.

Stop here so we can bring it back together and introduce the first activity.

---

# Activity 1: Plotting and exploring tabular data

In this notebook, we will visualize the table of confirmed exoplanets from the [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/index.html) (as of 30th September 2019), and we will take a look at some of the more advanced features of tabular data visualization.

## Loading the tabular data

We start off by loading the table using the [astropy.table](https://docs.astropy.org/en/stable/table/) module:

In [ ]:
from astropy.table import Table
exoplanets = Table.read(datapath('planets_2019.09.30_16.25.03.votable'))

Here's a helpful technique: take a look at the first 5 rows of the table to get a sense of what the data look like.

In [ ]:
exoplanets[:5]

How many rows are there in total?

In [ ]:
len(exoplanets)

## Adding a tabular data layer

WWT has a model of different "layers" that add data to the view. Once we have an Astropy data table, adding it to the view is easy. We start off by calling `add_table_layer` to create the layer:

In [ ]:
layer = wwt.layers.add_table_layer(exoplanets)

By default the points are quite small, so we can enlarge them:

In [ ]:
layer.size_scale = 50

You should now see points appear in the widget above. Have a look around and see what the distribution of these confirmed exoplanets is on the sky. For example, you can use the following to point the view at the Kepler field:

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord
wwt.center_on_coordinates(SkyCoord(76.32, 13.5, unit='deg', frame='galactic'), fov=20 * u.deg)

## Color-coding and scaling of points by attributes

All the points are currently white, but you can also choose to color code points by one of the other attributes. For example, we can color code the points by effective temperature of the parent star:

In [ ]:
layer.cmap_att = 'st_teff'

The ``cmap_vmin`` and ``cmap_vmax`` properties can be used to specify which values should be used as the extremes for the colorbar:

In [ ]:
layer.cmap_vmin = 5000
layer.cmap_vmax = 8000

and finally the colormap can be set using the ``cmap`` attribute, which takes either the name of a [Matplotlib colormap](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html) or a Matplotlib colormap object:

In [ ]:
layer.cmap = 'RdYlBu'

We can also scale the size of the points by one of the attributes — for example by the mass of the planet, and similarly we can set lower and upper values for the scaling:

In [ ]:
layer.size_att = 'pl_bmassj'
layer.size_vmin = 0
layer.size_vmax = 30

## Visualizing the data in 3D

Luckily, the exoplanet catalog contains *Gaia* distances to most of the planetary systems, so we can now tell the layer about the third dimension:

In [ ]:
layer.alt_att = 'gaia_dist'
layer.alt_type = 'distance'

Here's an important step. WWT includes a 3D mode that lets us explore this data table in, well, 3D:

In [ ]:
wwt.set_view('solar system')

Start zooming out, with a two finger scroll, using your mouse wheel, or by pressing 'x'. It will take a while, but eventually you'll see 3D motion of the points from our data table. Some of the larger points dominate the view, so we can turn off the scaling by size:

In [ ]:
layer.size_att = ''

Keep on zooming out until you can see (an artist’s rendering of) the plane of the Milky Way. At this point, if your orient the view in the right 3D direction, you can see a cone of systems which corresponds to those discovered in the original Kepler field!

**Note**: in Firefox, this stage can briefly slow down your browser a lot. Let WWT grind through for a minute and it will become smooth again.

You may notice that some points appear and disappear as you rotate around. To avoid this effect, set the following option:

In [ ]:
layer.far_side_visible = True

When this option is ``False`` (the default), points on the other side of the Sun (or the reference object being used for plotting) are hidden — this is useful when plotting on e.g. planets and the Sun, but not when looking at this kind of data.

When you're done playing around with this view, reset the widget so that we can begin the next stage of the tutorial:

In [ ]:
wwt.set_view('sky')
wwt.reset()

## Tabular data with a time-series axis

We like to say that WWT is a "4D" engine because it also knows about the passage of time. In this section, we'll demonstrate this functionality using a table of GRB observations.

First, we'll need some imports:

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.time import Time, TimeDelta
from astropy import units as u
from datetime import datetime

Now, we'll load up a table of GRB data. As before, we'll print out the first few rows to get a sense of the data structure. In this case it's pretty simple.

In [ ]:
bursts = Table.read(datapath('grb_table_lite.ecsv'), format='ascii.ecsv')
bursts[:5]

The WWT engine doesn't know enough to know that the Julian Date values above are actually dates (and not just some random floating-point measurements), so we convert them to the ISO8601 format:

In [ ]:
bursts['dates_ISOT'] = Time(bursts['dates_JD'], format='jd').isot
bursts[:5]

As before, let's create a table layer:

In [ ]:
layer = wwt.layers.add_table_layer(table=bursts, frame='Sky', lon_att='ra', lat_att='dec', size_scale=200)

Now let's tell the engine that we want to pay attentio to the time axis of the table. **This will make all of the points disappear**, since the engine's clock is synchronized to your computer — it's plotting data for the year 2019, and all of our GRBs are much older than that.

In [ ]:
layer.time_series = True
layer.time_att = 'dates_ISOT'

We can check the latter assertion using the `get_current_time()` method:

In [ ]:
wwt.get_current_time()

Let's demonstrate that things are actually working by centering on the first GRB and setting the time to just before it goes off.

In [ ]:
first_grb = bursts[0]

wwt.center_on_coordinates(SkyCoord(first_grb['ra'], first_grb['dec'], unit=u.deg))
wwt.set_current_time(Time(first_grb['dates_ISOT'], format='isot') - TimeDelta(3 * u.second))

You should see a white dot appear in the screen center after three seconds!

The WWT timeseries support has the concept of a "decay time". Points that appear on the screen slowly fade over the decay timescale — according to the WWT simulation clock, that is, not wallclock time. The default is 16 days. That's a bit short for visualizing GRBs, so let's make it longer:

In [ ]:
layer.time_decay = 1 * u.yr

Now, let's speed time up to see the GRBs pop off over time. If you find the right patch of the sky (namely, the ecliptic), you might also see the Sun and planets moving around at enhanced speed.

In [ ]:
wwt.play_time(rate=3e6)

Once you've had your fill, reset the widget again:

In [ ]:
wwt.play_time()  # this resets the clock to run at 1 second per second
wwt.reset()

Stop here so that we can bring it back together, ask questions, and discuss how things went.

---

# Activity 2: Displaying and Manipulating Image Data

In this activity, we will visualize a WISE 12µm image towards the [Westerhout 5 star forming region](https://en.wikipedia.org/wiki/Westerhout_5), and we will take a look at some of the advanced visualization options.

Images, like data tables, are represented in WWT as "layers" that can be added to the view. With a standard FITS file, all you need to do is provide a pathname:

In [ ]:
layer = wwt.layers.add_image_layer(datapath('w5.fits'))

The viewer will automatically center and zoom to the image you've loaded. You may get a warning from the `reproject` module; this can safely be ignored.

"Printing" the following variable will create a set of widgets that let you adjust how the data are visualized:

In [ ]:
layer.controls

The image color scaling is controlled by the sliders in the "Fine min/max" row; the "Coarse min/max" boxes control the bounds that are placed on the range of those sliders.

You should try sliding the image opacity back and forth to check the agreement between the morphology of the W5 image and the WWT all-sky map.

All of the parameters that are controlled by the widgets above can be manipulated programmatically as well. Let's set a bunch of them at once:

In [ ]:
layer.cmap = 'plasma'
layer.vmin = 400
layer.vmax = 1000
layer.stretch = 'sqrt'
layer.opacity = 0.9

Note that the settings in the widgets adjusted automatically to match what you entered. Fancy!

After you're done playing around, let's reset the WWT widget as usual:

In [ ]:
wwt.reset()

Let's pause for the next segment.

---

# Activity 3: pywwt as Part of an Ecosystem

Because pywwt is a Python module, not a standalone application, it gains a lot of power by being able to integrate with other components of the modern, Web-oriented astronomical software ecosystem.

## Loading data from remote sources

For instance, it is easy to use the Python module [astroquery](https://astroquery.readthedocs.io/en/latest/) to load in data directly from archive queries, without the requirement to save any files locally. Let's fetch 2MASS Ks-band images of the field of supernova 2011fe:

In [ ]:
from astroquery.skyview import SkyView

img_list = SkyView.get_images(position='SN 2011FE', survey='2MASS-K', pixels=500)  # you can adjust the size if you want
assert len(img_list) == 1  # there's only one matching item in this example
twomass_img = img_list[0]
twomass_img.info()

We can import it into WWT in the usual way:

In [ ]:
twomass_layer = wwt.layers.add_image_layer(twomass_img)

Once again you should see the view automatically center on your image. Let's adjust the background imagery to be more relevant:

In [ ]:
wwt.background = wwt.imagery.ir.twomass
wwt.foreground_opacity = 0

We also provide interactive controls to let you adjust the parameters of the contextual imagery that's being shown. Try choosing different sets of all-sky imagery and adjusting the blend between them:

In [ ]:
wwt.layer_controls

Here are some settings that we like:

In [ ]:
wwt.background = wwt.imagery.visible.sdss
wwt.foreground = wwt.imagery.gamma.fermi
wwt.foreground_opacity = .7

Now we'll load up another image of the same field that came from *Swift*, this time stored as a local file as in our previous activity:

In [ ]:
swift_layer = wwt.layers.add_image_layer(datapath('m101_swiftx.fits'))

Create controls to adjust all of the visualization parameters. If you want to go wild, you can overlay data from four different wavelengths in this one view!

In [ ]:
wwt.layer_controls

In [ ]:
twomass_layer.controls

In [ ]:
swift_layer.controls

# Exporting to standalone HTML

Pywwt, like all Jupyter widgets, is at its heart web-based. We've been working on building the technology to extract the visualizations that you create from Jupyter into standalone web assets that can be included anywhere that you can host HTML + JavaScript + CSS. In particular, we aim to support these assets as [interactive figures](https://journals.aas.org/graphics-guide/#interactive_figures) in the AAS journals.

Exporting everything is easy, but due to an unfortunate bug in our JupyterLab support, it doesn't work reliably when you separate out the WWT window as have done in this tutorial. So, let's create a secondary WWT window:

In [ ]:
wwt2 = WWTJupyterWidget()
wwt2

For what it's worth, this does demonstrate that you can create multiple WWT widgets in the same notebook!

Now, let's repopulate it with our images:

In [ ]:
twomass_layer2 = wwt2.layers.add_image_layer(twomass_img)
swift_layer2 = wwt2.layers.add_image_layer(datapath('m101_swiftx.fits'), opacity=0.5, cmap='plasma')

You could tweak the appearance here by inserting cells to create control widgets with `wwt2.layer_controls`, `twomass_layer2.controls`, and `swift_layer2.controls`.

Once we've jumped through this hoop, exporting the HTML is this easy:

In [ ]:
wwt2.save_as_html_bundle('figure.zip')

You should see a file named `figure.zip` appear in the folder view sidebar to the left of this notebook. (It might take a few seconds to appear.) You can right-click that file and select "Download" to save the Zip bundle to your local machine.

You might want to open up this Zip file and inspect its contents. You will see that it contains an `index.html` file, some JavaScript, and a `data` directory containing two FITS files. These correspond to the images that you just loaded into your view.

If you have [Node.js](https://nodejs.org/en/) installed on your computer, you can view your exported HTML in a straightforward way:

1. Unpack the Zip file
2. In a terminal, `cd` to the unpacked directory (the one containing `index.html`).
3. Run: `npx httpserver 12345`
4. Open a browser to the URL printed by the program.

**Note**: alas, we *also* have a race condition bug that sometimes pops up here. You might need to reload the page for the images to appear.

Other one-liner web servers might work, but not all of them serve file with the correct `Content-Type` headers, which is needed for everything to work here.

Let's pause here again.

---

# Activity 4: Free Exploration!

We will end with some time for free exploration! Here are some pointers that we hope are useful.

## Have your own data?

If you have your own data that you would like to explore, you can upload the files to the BinderHub server so that they can be accessed by the cloud-based notebooks. Just click the underlined-up-arrow icon in the Folder panel to the left of this notebook.

## Don't have your own data?

If not, take a look at the [Astroquery Gallery of Queries](https://astroquery.readthedocs.io/en/latest/gallery.html) for some queries that you can get started with.

## Documentation

Some of this documentation might be helpful:

- [pywwt](https://pywwt.readthedocs.io/)
- [astroquery](https://astroquery.readthedocs.io/)
- [astropy](https://docs.astropy.org/en/stable/)

## Data too big?

We've used the cloud-based notebooks for convenience, but installing pywwt locally is not that hard. The recommended mechanism is through Anaconda. In a temporary environment, run:

```
conda install -c conda-forge pywwt
```

In this case it may be easier to use plain Jupyter notebooks, rather than Jupyter Lab, since setting up the Jupyter Lab widget system can sometimes be finicky.

---

# What's Next?

Want to get more involved in the AAS WorldWide Telescope community? Here are some next steps you can take:

- Sign up for our mailing list: <https://bit.ly/wwt-signup>
- Join our discussion forum: <https://wwt-forum.org/>
- Visit the WWT Contributor Hub: <https://worldwidetelescope.github.io/>
- Follow us on social media:
  - [@wwtelescope](https://twitter.com/wwtelescope) on Twitter
  - [AASWorldWideTelescope](http://www.youtube.com/c/AASWorldWideTelescope) on YouTube
  - [@wwtelescope](http://www.youtube.com/c/AASWorldWideTelescope) on Facebook
- We have [other bonus links on the tutorial thread](https://wwt-forum.org/t/researcher-workshop-wwt-adass29-groningen-netherlands-2019-october-6/70/3)

We'll have plenty of activities at [AAS 235](http://www.youtube.com/c/AASWorldWideTelescope) if you'll be in Hawai'i in January!

# Credits

The examples in this notebook were prepared by:

- O. Justin Otor
- Thomas Robitaille
- Peter K. G. Williams

With thanks to Mark SubbaRao for creating the first version of the GRB example notebook.